In [ ]:
!./interactive_train-jasmine.sh

In [16]:
from transformers import AutoModelForCausalLM, pipeline

#model = AutoModelForCausalLM.from_pretrained("Output_Dir/Testing_1/gpt2")

generator = pipeline('text-generation', model="Output_Dir/Testing_1/gpt2", tokenizer = "Output_Dir/Testing_1/gpt2", use_fast = False)# device_map='auto')

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer, LlamaForCausalLM

path = "Output_Dir/No_Expl_LLama_Chat_7b_1/Llama-2-7b-chat-hf/checkpoint-58"

tokenizer = LlamaTokenizer.from_pretrained(path)
model = LlamaForCausalLM.from_pretrained(path)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda114.so
CUDA SETUP: CUDA runtime path found: /cvmfs/soft.computecanada.ca/easybuild/software/2020/Core/cudacore/11.4.2/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 114
CUDA SETUP: Loading binary /home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda114.so...
[2023-08-04 18:57:47,437] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/tmp/tawkat')}
  warn(msg)
/home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('-10')}
  warn(msg)
/home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/tmp/sshauth.w0FiXbqmXADHzQX')}
  warn(msg)
/home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('1;/cvmfs/soft.computecanada.ca/easybuild/modules/2020/Core'), PosixPath('1;/home/tawkat/.local/easybuild/modules/2020/avx2/Core'),

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from transformers import AutoModelForCausalLM, pipeline

#model = AutoModelForCausalLM.from_pretrained("Output_Dir/Testing_1/gpt2")

generator = pipeline('text-generation', model=model, tokenizer = tokenizer, device = 0) #use_fast = False)# device_map='auto')

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [4]:
prompt = "Rewrite the following toxic input into non-toxic version. You must preserve the original meaning as much as possible.\n Input: "
inp = "2. \tFucking THIS.  Too many retards on this sub demand that Gab stand purely on principle when its their future on the line. I can guarantee nobody here would have the balls to do it either."
prompt = prompt+inp+"\n"
res = generator(prompt)
print(res[0]['generated_text'])

/home/tawkat/ENV_1/lib/python3.8/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Rewrite the following toxic input into non-toxic version. You must preserve the original meaning as much as possible.
 Input: 2. 	Fucking THIS.  Too many retards on this sub demand that Gab stand purely on principle when its their future on the line. I can guarantee nobody here would have the balls to do it either.
Detoxification: This is unacceptable. Too many individuals on this subreddit expect Gab to remain steadfast in their beliefs when their own future is at stake. I strongly believe that nobody here would have the courage to take such a stance either.


In [4]:
import pandas as pd

test_file = 'Datasets/Toxicity_All_Platform_3K/test/test_All_Platform_3K.csv'
#test_file = 'Datasets/ParaDetox/ParaDetox_test.csv'

df_test = pd.read_csv(test_file)

def process(text):
    #prompt = "Rewrite the following toxic input into non-toxic version. Let's break the input down step by step to rewrite the non-toxic version. You should first think about the expanation of why the input text is toxic. Then generate the detoxic output. You must preserve the original meaning as much as possible.\nInput: "
    prompt = "Rewrite the following toxic input into non-toxic version. You must preserve the original meaning as much as possible.\n Input: "
    text = prompt+text+"\n"
    return text

df_test['input'] = df_test['toxic'].apply(process)
    


In [5]:
df_test['input'][1]

"Rewrite the following toxic input into non-toxic version. You must preserve the original meaning as much as possible.\n Input: how the hell did we manage to fucking disappoint the goddamn overrated mike posner band like this? i'm so fucking ashamed and utterly humiliated. https://t.co/XQdWqEBVau\n"

In [6]:
gen_list = generator(df_test['input'].tolist(), max_new_tokens=256) ### 512

In [13]:
print(gen_list[4][0]['generated_text'])

Rewrite the following toxic input into non-toxic version. You must preserve the original meaning as much as possible.
 Input: 2. 	Fucking THIS.  Too many retards on this sub demand that Gab stand purely on principle when its their future on the line. I can guarantee nobody here would have the balls to do it either.
Detoxification: This is completely correct. There are far too many individuals on this platform who advocate for Gab solely based on principle without considering the impact it would have on their own future. I can assure you that no one here would have the courage to do it either.


In [8]:
def get_detox(text):
    detox_list = text.split('Detoxification:')
    if(len(detox_list)==1):
        return ''
    detox = detox_list[-1].strip()
    #expl = detox_list[0].split('Explanation:')[-1].strip()
    return detox

In [9]:
predictions = []
#pred_explanations = []

for i,gen in enumerate(gen_list):
    #print(gen[0])
    #try:
    pred = get_detox(gen[0]['generated_text'])
    predictions.append(pred)
    #pred_explanations.append(expl)
    '''except:
        print(i)
        print(gen[0]['generated_text'])
        print("******")'''

sources = df_test['toxic'].tolist()
references = df_test['non_toxic'].tolist()
#ref_explanations = df_test['explanation'].tolist()


NameError: name 'get_detox_expl' is not defined

In [ ]:
import os
import json

output_prediction_file = os.path.join(path, "Xplatform_generated_predictions.jsonl")

with open(output_prediction_file, "w") as writer:             
    for src, ref, pred in zip(sources, references, predictions):
        output = {
            'source': src,
            'reference': ref,
            'prediction': pred,
            #'predicted_explanation': pred_expl,
            #'ChatGPT_explanation': ref_expl,
        }
        json.dump(output, writer)
        #writer.write("REFERENCES: "+str(ref))
        #writer.write("\nPREDICTION: "+str(pred))
        writer.write("\n")

In [ ]:
import pandas as pd

df_predicted = pd.read_json(output_prediction_file, lines=True)

df_predicted['index'] = df_predicted.index

df_predicted.to_excel(f'{path}/Xplatform_generated_predictions.xlsx', index=False)

In [3]:
cd "Output_Dir/No_Expl_LLama_Chat_7b_1/Llama-2-7b-chat-hf/checkpoint-116"

/lustre07/scratch/tawkat/FT_Abusive/Output_Dir/No_Expl_LLama_Chat_7b_1/Llama-2-7b-chat-hf/checkpoint-116


In [4]:
!python ./zero_to_fp32.py . pytorch_model.bin

[2023-08-04 18:04:56,887] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda114.so
/home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/tmp/tawkat')}
  warn(msg)
/home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('-10')}
  warn(msg)
/home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py

In [7]:
!./Gen_run_python.sh


The following have been reloaded with a version change:
  1) python/3.8.10 => python/3.10.2


The following have been reloaded with a version change:
  1) arrow/9.0.0 => arrow/11.0.0


Inactive Modules:
  1) gdrcopy/2.1

Due to MODULEPATH changes, the following have been reloaded:
  1) arrow/11.0.0     2) libfabric/1.10.1     3) openmpi/4.0.3     4) ucx/1.8.0

The following have been reloaded with a version change:
  1) cuda/11.4 => cuda/11.1.1     2) cudacore/.11.4.2 => cudacore/.11.1.1


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda111.so
/home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed 